### Budget trends in RL data

The following code uses the `all_activity_normal.cs` dataset to create trends lines for average advertising budget for the 2009-2018 period. The dataset can be found at `G:\Market\for Matt & B2B\Data\ReachLocal\Platform Data\all_activities 2009-Nov. 2018.zip` The code can be modified to create these graphs based on a variety of categories, for example business categoty or advertiser subcategory. The logic is fairly straight forward and following the comments along should be easy. The code has been automated to some extend and will be updated in the future to a turn-key code.

Author suggets that the following hierarchy is employed to save the everything:

Main folder/ >>
- /code
- /data
- /output

*Author: Farshad Nasiri - B2B Data Science Team*

In [2]:
# Import necessay libraries

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

#################################################################
## For including LateX interpreter
#################################################################
#from matplotlib import rc
#rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
#rc('text', usetex=True)

In [3]:
# Read the data
df=pd.read_csv('../data/all_activity_normal.csv')

In [4]:
# Quick look at the shape of the dataframe
df.shape

(1957753, 55)

In [5]:
# Quick look at the columns
df.columns

Index(['Unnamed: 0', 'idcampaign', 'idcampaign_master', 'idOffer',
       'Offer_Name', 'Finance_Product', 'idadvertiser', 'idadvertiser_master',
       'Advertiser_URL', 'idBusiness', 'Channel', 'Country', 'Currency',
       'campaign_budget', 'Retail_Cost', 'COGS', 'Overage', 'impressions',
       'clicks', 'CVTs', 'qualified_calls', 'calls', 'emails',
       'qualified_web_events', 'Leads', 'CPC', 'CTR', 'CPL', 'CTL', 'Paid_On',
       'Cycle_Started', 'Cycle_Ended', 'MCID_Started', 'Cycle_Number',
       'Advertiser_BC_ID', 'Advertiser_BusinessCategory', 'Advertiser_BSC_ID',
       'Advertiser_BusinessSubCategory', 'BC_ID', 'BusinessCategory',
       'Primary_BSC_ID', 'Primary_BusinessSubCategory', 'Secondary_BSC_Count',
       'Secondary_BSC_IDs', 'Seconardy_BSCs', 'BusinessSpecialtyID',
       'BusinessSpecialty', 'TargetType', 'Radius_Target_Address',
       'Radius_Target_Miles', 'Cities_Targetd', 'DMAs_Targeted',
       'Population_Target', 'ratio', 'ratio_cat'],
      dtype='

In [6]:
# Summary of missing values
df.isnull().sum()

Unnamed: 0                              0
idcampaign                              0
idcampaign_master                       0
idOffer                                 0
Offer_Name                            275
Finance_Product                       275
idadvertiser                            0
idadvertiser_master                     0
Advertiser_URL                      35884
idBusiness                             11
Channel                                11
Country                                11
Currency                               11
campaign_budget                         0
Retail_Cost                             0
COGS                                    0
Overage                                 0
impressions                             0
clicks                                  0
CVTs                                    0
qualified_calls                         0
calls                                   0
emails                                  0
qualified_web_events              

In [7]:
# Convert the date columns to date-time format
df['Cycle_Started']=pd.to_datetime(df.Cycle_Started)
df['Cycle_Ended']=pd.to_datetime(df.Cycle_Ended)

In [8]:
# Remove the one row with campaign start date missing
# and reset the index
df=df[~df.Cycle_Started.isnull()]
df.reset_index(drop=True, inplace=True)

In [9]:
# Drop the columns that we won't be using for graph generation 
# in order to reduce memory load

df.drop(columns=['Unnamed: 0', 'Primary_BSC_ID', 'BusinessSpecialtyID', 'Advertiser_URL', 
                 'Country','Currency','BusinessSpecialty', 'CPC', 'CTR', 'CPL', 'CTL', 
                 'Paid_On'], axis=1, inplace=True)
df.reset_index(drop=True, inplace=True)

In [10]:
# We create the graphs here with the follwoing logic.
# First we choose a category to collect data on. So for exmple if we 
# were to choose `BusinessCategory` then the resulting plots will be
# the trend lines for different business categories. The goal is to 
# create a new dataframe that looks like this:
#                     2009Q1   2009Q2  ...  2018Q4
# index            -------------------------------
# ----------------|          |        |... |      |
# Animals and pets|          |        |... |      |
# Furniture       |          |        |... |      |
# Real Estate     |          |        |... |      |
# ...

# We call this dataframe `my_df`.
# The average budget across all business in animals and pets in 2009Q1 
# will go into the first element of the dataframe, so on and so forth.


col='BusinessCategory' # Choose a category


df[col].fillna('Other', inplace=True) # Fill missing values with "Other"

# Create a temporay dataframe to save the following columns:
# col, 'campaign_budget','Cycle_Started','Cycle_Ended'
# It'll be smaller than the main dataframe and easier to use

temp=df[[col, 'campaign_budget','Cycle_Started','Cycle_Ended']] 

# Get all the unique values of that column. This will be the index
names=df[col].unique() 
row_size=len(names) # Number of rows in my_df

years=range(2009,2019) # The time span in years

# Creating the column names for my_df
col_names=[]
for year in years:
    for quarter in [1,2,3,4]:
        col_names.append(str(year)+'-Q'+str(quarter))

# Creating my_df and filling it with zeros
my_df=pd.DataFrame(data=np.zeros((row_size,4*len(years))), columns=col_names)

# Sorting the index for my_df alphabetically
my_df.index=sorted(names.tolist())


for quarter in range(1,5): # Loop through quarters and define the periods within a year
    if quarter==1:
        start_str='-01-01'
        end_str='-03-31'
    if quarter==2:
        start_str='-04-01'
        end_str='-06-30'
    if quarter==3:
        start_str='-07-01'
        end_str='-09-30'
    if quarter==4:
        start_str='-10-01'
        end_str='-12-31'
    
    for year in years: # Loop through years
        start=pd.to_datetime(str(year)+start_str) # Create start date for a specific quarter
        end = pd.to_datetime(str(year)+end_str)   # Create end date for a specific quarter
        
        # Slice the temporary dataframe with the start-end dates created above
        # and store in temp2
        temp2=temp[(temp.Cycle_Started >= start) & (temp.Cycle_Ended <= end)]
        
        # Group by the values in the `col` (e.g. Furniture etc.) and aggregate with mean
        temp2=temp2.groupby(by=[col]).agg({'campaign_budget':'mean'})
       
        # Sort the values so that they would align with the indecies of my_df
        temp2.sort_values(by=[col], inplace=True)
        
        # Store in the corresponding column in my_df
        my_col=str(year)+'-Q'+str(quarter)
        my_df.loc[temp2.index, my_col]=temp2.campaign_budget

In [11]:
# Take a look
pd.set_option("display.max_columns",100)

my_df

,2009-Q1,2009-Q2,2009-Q3,2009-Q4,2010-Q1,2010-Q2,2010-Q3,2010-Q4,2011-Q1,2011-Q2,2011-Q3,2011-Q4,2012-Q1,2012-Q2,2012-Q3,2012-Q4,2013-Q1,2013-Q2,2013-Q3,2013-Q4,2014-Q1,2014-Q2,2014-Q3,2014-Q4,2015-Q1,2015-Q2,2015-Q3,2015-Q4,2016-Q1,2016-Q2,2016-Q3,2016-Q4,2017-Q1,2017-Q2,2017-Q3,2017-Q4,2018-Q1,2018-Q2,2018-Q3,2018-Q4
Animals & Pets,706.474818,694.401188,764.180627,758.376134,779.268128,744.725012,852.532805,858.523614,794.092984,802.845691,818.575311,820.541056,819.436810,826.851003,832.761624,799.295115,828.578577,832.225071,864.719247,823.346565,781.163406,791.859826,782.973937,807.019646,761.479079,781.584475,787.891827,795.233233,806.676638,762.923477,754.825459,751.311989,749.429487,739.861343,724.559954,812.967692,781.688521,730.914884,689.663272,676.510602
Apparel / Fashion & Jewelry,1069.668831,1133.314983,1005.196477,1085.459944,1328.143952,1217.345995,1163.863003,1283.564444,1224.693780,1299.046062,1410.678894,1372.173928,1421.738095,1453.199786,1326.772065,1295.442890,1311.246407,1308.786290,1241.897087,1250.856680,1085.743590,1018.991777,973.396783,1038.323204,986.350820,958.543960,980.288951,953.856102,870.791629,918.783971,880.965364,910.276928,990.215534,965.661714,971.059183,937.960186,985.104438,912.167874,958.628743,1098.184615
Arts & Entertainment,1126.727037,961.799865,1086.953175,1130.383047,1141.546875,1159.226635,1114.809129,1066.657525,1093.524730,1164.200169,1185.399241,1264.822297,1252.585926,1166.449868,1210.965000,1171.723467,1135.537590,1122.194747,1113.605249,1202.905299,1173.518718,1355.461814,1320.720479,1325.354286,1349.381535,1478.580243,1488.602802,1393.440366,1426.480000,1409.173622,1622.721507,1630.956070,1722.753370,1753.312881,1473.350816,1461.084569,1461.570027,1149.895986,1557.329233,1822.590763
Attorneys & Legal Services,1489.769172,1582.664342,1619.651953,1624.176536,1731.744369,1904.348128,1895.669142,1779.469788,1716.517570,1775.181025,1750.693961,1777.617553,1768.672329,1757.586771,1860.631502,1718.753720,1749.048374,1698.482038,1750.831160,1774.888722,1816.652479,1687.224940,1479.116028,1500.196678,1551.704785,1494.339707,1492.354103,1416.828925,1364.740704,1534.699497,1505.763789,1380.384814,1258.927703,1274.723758,1292.856112,1382.520798,1412.424407,1402.400017,1322.125672,1364.485948
Automotive -- For Sale,1681.542029,1730.101940,1794.438489,1879.378385,1918.869286,1875.932564,1886.415966,1963.042292,1969.322402,1928.080471,1898.090062,1898.459507,1933.301395,1938.199917,1870.943842,1856.447417,1824.006417,1842.194016,1882.208131,1858.555261,1952.166868,1983.959497,1895.601392,1733.024539,1877.873673,1895.269260,1905.728734,1882.734014,1811.874723,1846.486096,1801.346245,1768.488700,1740.355222,1793.396317,1743.012116,1816.995928,1727.700164,1698.816617,1664.730907,1646.318649
"Automotive -- Repair, Service & Parts",865.869808,950.710645,857.061167,910.269921,880.589384,849.240259,859.892185,892.214179,911.696001,885.016635,883.796136,890.948692,931.739227,994.966519,1176.408523,977.187719,1020.159002,1047.650321,1005.398397,1006.312663,958.980023,995.200647,1003.597453,988.779413,956.727393,957.581058,913.882457,935.291331,921.923920,945.643385,959.987037,926.177594,962.083815,832.898072,815.433068,869.662100,921.477077,901.741584,894.200132,880.573843
Beauty & Personal Care,1207.531264,1197.095257,1227.119826,1155.502596,1183.509683,1226.520779,1233.947213,1261.699050,1267.727346,1233.481440,1284.639651,1168.559267,1298.164413,1252.623063,1202.937773,1262.868830,1262.692295,1149.193513,1122.272407,1120.019499,1054.487174,1000.270544,1006.994487,1066.974232,1027.280820,1082.091810,976.477527,1092.555187,1143.480532,1118.053062,1105.783937,1162.924330,1041.125000,982.690332,936.241620,898.735452,948.346111,972.521029,1003.867986,859.087288
Business Opportunities,1247.142857,1086.666667,1289.473684,1259.256667,972.991667,1193.623448,3305.150000,2834.385965,2066.106061,3443.847826,3266.326087,2303.804348,4055.086957,2021.708333,1264.500000,1123.500000,1247.222222,917.842105,111

In [13]:
# Creating plots

# Where to store the graphs
path = '../output/'+col.replace(' ', '_')

# Create the folder if it doesn't exist
os.makedirs(path, exist_ok=True)

for i in range(0, len(my_df.index)):
    location=i

    plt.figure(figsize=(12,6))
    plt.cla()
    plt.plot(my_df.columns, my_df.iloc[location,:],'k-o')
    plt.xticks(rotation='vertical');
    plt.tick_params(direction='in', length=10, width=2.5, colors='k',
                               grid_color='k', grid_alpha=1, labelsize=14)
    plt.ylabel('Ave. Budget', fontsize=16)
    plt.title(my_df.index[location], fontsize=16)
    #ax = plt.gca()
    
    save_to=path+'/'+my_df.index[location].replace(' ', '_').replace('/','')+".png"
    plt.savefig(save_to, bbox_inches = 'tight')
    plt.close()